# Comprehensive Weekly Forecast Update - FULL RUN (All 18 Models)

**Purpose**: Complete weekly update with ALL forecasting models

## What This Notebook Does:
1. **Fetch Actuals** - Query last week's actual shipments from Databricks
2. **Evaluate ALL 18 Models** - Run comprehensive model comparison
3. **Update Routing** - Determine winning models and update routing table
4. **Track Performance** - Store results in performance tracking database
5. **Generate Forecast** - Create current week's forecast using optimal models
6. **Export Files** - Save all CSVs and summaries

## Models Included (18 Total):

**Traditional Models (13):**
- 01_Historical_Baseline, 02_Recent_2W, 03_Recent_4W_HYBRID, 04_Recent_8W
- 05_Trend_Adjusted, 06_Prior_Week, 07_Same_Week_Last_Year, 08_Week_Specific
- 09_Exp_Smoothing, 10_Probabilistic, 11_Hybrid_Week_Blend
- 12_Median_Recent, 13_Weighted_Recent_Week

**Advanced Models (5):**
- 14_SARIMA (Time series), 15_ML_Classifier, 16_ML_Regressor
- 17_Lane_Adaptive, 18_Clustering

## Runtime: 30-60 minutes

⏱️ SARIMA model fitting is the bottleneck (~30-40 min for 1,500 routes)

## How It Works:

**Weeks are automatically calculated:**
- **Evaluation Week**: Last week (current week - 1) - has actuals
- **Forecast Week**: Current week - what we're forecasting for

Example (running on Dec 15, 2025):
- Current week: 51
- Evaluation week: 50 (last week, has actuals)
- Forecast week: 51 (this week)

**To override**, manually set `EVALUATION_WEEK` and `FORECAST_WEEK` in the configuration cell.

Run all cells to update everything!

## Setup and Configuration

In [ ]:
# Imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from databricks import sql
import sqlite3
import json
import warnings
warnings.filterwarnings('ignore')

# Progress bars
from tqdm.auto import tqdm

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

# Import model functions
from forecast_comprehensive_all_models import ComprehensiveModels

print("✅ Setup complete!")

In [ ]:
# ========== CONFIGURATION ==========
# Automatically calculate weeks based on current date

today = datetime.now()
current_week = today.isocalendar()[1]  # Get ISO week number
current_year = today.year

# Evaluation week = last week (week that just ended, has actuals)
EVALUATION_WEEK = current_week - 1

# Forecast week = current week (the week we're in now)
FORECAST_WEEK = current_week

# Handle year boundary (if we're in week 1, evaluation week is last week of previous year)
if EVALUATION_WEEK < 1:
    EVALUATION_WEEK = 52  # Last week of previous year
    EVALUATION_YEAR = current_year - 1
    FORECAST_YEAR = current_year
else:
    EVALUATION_YEAR = current_year
    FORECAST_YEAR = current_year

# Databricks connection
DATABRICKS_CONFIG = {
    "server_hostname": "adb-434028626745069.9.azuredatabricks.net",
    "http_path": "/sql/1.0/warehouses/23a9897d305fb7e2",
    "auth_type": "databricks-oauth"
}

TABLE_NAME = "decus_domesticops_prod.dbo.tmp_hassett_report"

# Output timestamp
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

print(f"📋 Auto-Configuration (Today: {today.strftime('%Y-%m-%d')}):")
print(f"  Current Week: {current_week}")
print(f"  Evaluation Week: {EVALUATION_WEEK}, {EVALUATION_YEAR} (last week - has actuals)")
print(f"  Forecast Week: {FORECAST_WEEK}, {FORECAST_YEAR} (current week)")
print(f"  Timestamp: {TIMESTAMP}")
print(f"\n💡 Note: Weeks are auto-calculated. To override, manually set EVALUATION_WEEK and FORECAST_WEEK.")

## Step 1: Connect to Databricks and Load Data

In [ ]:
# Connect to Databricks
conn = sql.connect(**DATABRICKS_CONFIG)
print("✅ Connected to Databricks")

# Load historical data (4 years lookback)
def load_historical_data(conn, target_week, target_year, table_name, years=4):
    year_start = datetime(target_year, 1, 1)
    target_date = year_start + timedelta(weeks=target_week - 1)
    lookback_date = target_date - timedelta(days=365 * years)

    query = f"""
    SELECT
        DATE_SHIP as date,
        ODC, DDC, ProductType,
        PIECES as pieces,
        weekofyear(DATE_SHIP) as week,
        YEAR(DATE_SHIP) as year,
        dayofweek(DATE_SHIP) as dayofweek
    FROM {table_name}
    WHERE DATE_SHIP >= '{lookback_date.strftime('%Y-%m-%d')}'
        AND DATE_SHIP < '{target_date.strftime('%Y-%m-%d')}'
        AND ProductType IN ('MAX', 'EXP')
        AND ODC IS NOT NULL
        AND DDC IS NOT NULL
    ORDER BY DATE_SHIP DESC
    """

    print(f"📊 Loading {years} years of historical data (up to week {target_week}, {target_year})...")
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    df['date'] = pd.to_datetime(df['date'])
    print(f"✅ Loaded {len(df):,} historical records")
    return df

# Load data up to evaluation week
df_historical = load_historical_data(conn, EVALUATION_WEEK, EVALUATION_YEAR, TABLE_NAME, years=4)

## Step 2: Get Actuals for Evaluation Week

In [ ]:
# Query actuals for the evaluation week
def get_week_actuals(conn, week, year, table_name):
    query = f"""
    SELECT 
        ODC,
        DDC,
        ProductType,
        dayofweek(DATE_SHIP) as dayofweek,
        SUM(PIECES) as actual_pieces
    FROM {table_name}
    WHERE weekofyear(DATE_SHIP) = {week}
        AND YEAR(DATE_SHIP) = {year}
        AND ProductType IN ('MAX', 'EXP')
    GROUP BY ODC, DDC, ProductType, dayofweek(DATE_SHIP)
    ORDER BY ODC, DDC, ProductType, dayofweek
    """
    
    print(f"📊 Querying actuals for week {week}, {year}...")
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    print(f"✅ Found {len(df):,} route-day actuals")
    print(f"   Total pieces: {df['actual_pieces'].sum():,.0f}")
    return df

df_actuals = get_week_actuals(conn, EVALUATION_WEEK, EVALUATION_YEAR, TABLE_NAME)

# Show sample
print("\nSample actuals:")
display(df_actuals.head(10))

## Step 3: Generate All Model Forecasts for Evaluation Week

In [ ]:
# Get all unique routes from actuals
routes = df_actuals[['ODC', 'DDC', 'ProductType', 'dayofweek']].drop_duplicates()
print(f"📊 Generating forecasts for {len(routes):,} route-days using ALL 18 MODELS...\n")
print(f"⏱️  Estimated time: 30-60 minutes (SARIMA is slow)\n")

# ALL 18 MODELS - Full comprehensive analysis
model_functions = [
    # Traditional models (13)
    ('01_Historical_Baseline', ComprehensiveModels.model_01_historical_baseline),
    ('02_Recent_2W', ComprehensiveModels.model_02_recent_2w_avg),
    ('03_Recent_4W_HYBRID', ComprehensiveModels.model_03_recent_4w_avg),
    ('04_Recent_8W', ComprehensiveModels.model_04_recent_8w_avg),
    ('05_Trend_Adjusted', ComprehensiveModels.model_05_trend_adjusted),
    ('06_Prior_Week', ComprehensiveModels.model_06_prior_week),
    ('07_Same_Week_Last_Year', ComprehensiveModels.model_07_same_week_last_year),
    ('08_Week_Specific', ComprehensiveModels.model_08_week_specific_historical),
    ('09_Exp_Smoothing', ComprehensiveModels.model_09_exponential_smoothing),
    ('10_Probabilistic', ComprehensiveModels.model_10_probabilistic),
    ('11_Hybrid_Week_Blend', ComprehensiveModels.model_11_hybrid_week_blend),
    ('12_Median_Recent', ComprehensiveModels.model_12_median_recent),
    ('13_Weighted_Recent_Week', ComprehensiveModels.model_13_weighted_recent_week),
    # Advanced models (5)
    ('14_SARIMA', ComprehensiveModels.model_14_sarima),
    ('15_ML_Classifier_Simple_Vol', ComprehensiveModels.model_15_ml_classifier_simple_volume),
    ('16_ML_Regressor', ComprehensiveModels.model_16_ml_regressor),
    ('17_Lane_Adaptive', ComprehensiveModels.model_17_lane_adaptive),
    ('18_Clustering', ComprehensiveModels.model_18_clustering),
]

# Generate forecasts for each route
results = []

# Progress bar
pbar = tqdm(routes.iterrows(), total=len(routes), desc="Evaluating all 18 models")
for idx, route in pbar:
    odc, ddc, product, dow = route['ODC'], route['DDC'], route['ProductType'], route['dayofweek']
    
    # Update progress bar with current route
    pbar.set_postfix({'Route': f"{odc}-{ddc}-{product}"})
    
    # Get route history
    route_data = df_historical[
        (df_historical['ODC'] == odc) &
        (df_historical['DDC'] == ddc) &
        (df_historical['ProductType'] == product) &
        (df_historical['dayofweek'] == dow)
    ].sort_values('date', ascending=False)
    
    # Get actual
    actual = df_actuals[
        (df_actuals['ODC'] == odc) &
        (df_actuals['DDC'] == ddc) &
        (df_actuals['ProductType'] == product) &
        (df_actuals['dayofweek'] == dow)
    ]['actual_pieces'].values[0] if len(df_actuals[
        (df_actuals['ODC'] == odc) &
        (df_actuals['DDC'] == ddc) &
        (df_actuals['ProductType'] == product) &
        (df_actuals['dayofweek'] == dow)
    ]) > 0 else 0
    
    result = {
        'route_key': f"{odc}|{ddc}|{product}|{dow}",
        'ODC': odc,
        'DDC': ddc,
        'ProductType': product,
        'dayofweek': dow,
        'Actual': actual
    }
    
    # Run each model
    for model_name, model_func in model_functions:
        try:
            forecast = model_func(route_data, EVALUATION_WEEK, EVALUATION_YEAR, product)
            result[model_name] = max(0, forecast)
        except Exception as e:
            # If model fails, use 0 (common for SARIMA with insufficient data)
            result[model_name] = 0
    
    results.append(result)

df_forecasts = pd.DataFrame(results)
print(f"\n✅ Generated forecasts for {len(df_forecasts):,} routes using all 18 models")

## Step 4: Calculate Errors and Find Winners

In [ ]:
# Calculate errors for each model
model_cols = [col for col in df_forecasts.columns if col not in ['route_key', 'ODC', 'DDC', 'ProductType', 'dayofweek', 'Actual']]

print(f"📊 Calculating errors for {len(model_cols)} models...")
print(f"   Models evaluated: {', '.join(model_cols)}\n")

for col in model_cols:
    error_col = f"{col}_Error%"
    df_forecasts[error_col] = np.where(
        df_forecasts['Actual'] > 0,
        abs((df_forecasts[col] - df_forecasts['Actual']) / df_forecasts['Actual'] * 100),
        np.where(df_forecasts[col] > 0, 999, 0)
    )

# Find winner for each route
error_cols = [col for col in df_forecasts.columns if col.endswith('_Error%')]
df_forecasts['Winner_Model'] = df_forecasts[error_cols].abs().idxmin(axis=1).str.replace('_Error%', '')
df_forecasts['Winner_Error%'] = df_forecasts[error_cols].abs().min(axis=1)

print(f"✅ Winners determined!")
print(f"\n🏆 Model Win Summary (Top 10):")
win_summary = df_forecasts['Winner_Model'].value_counts()
for i, (model, count) in enumerate(win_summary.head(10).items(), 1):
    pct = count / len(df_forecasts) * 100
    print(f"  {i:2}. {model:<30} {count:4,} routes ({pct:5.1f}%)")

## Step 5: Save Comprehensive Results

In [ ]:
# Save comprehensive results
output_file = project_root / f'comprehensive_all_models_week{EVALUATION_WEEK}.csv'
df_forecasts.to_csv(output_file, index=False)
print(f"💾 Saved: {output_file.name}")
print(f"   Size: {len(df_forecasts):,} routes x {len(df_forecasts.columns)} columns")

## Step 6: Update Routing Table

In [ ]:
# Create routing table with winners
routing_table = df_forecasts[[
    'route_key', 'ODC', 'DDC', 'ProductType', 'dayofweek',
    'Winner_Model', 'Winner_Error%', 'Actual'
]].copy()

routing_table.columns = [
    'route_key', 'ODC', 'DDC', 'ProductType', 'dayofweek',
    'best_model', 'best_error', 'actual'
]

# Add confidence levels
def assign_confidence(error):
    if error == 999:
        return 'LOW'
    elif error <= 20:
        return 'HIGH'
    elif error <= 50:
        return 'MEDIUM'
    else:
        return 'LOW'

routing_table['confidence'] = routing_table['best_error'].apply(assign_confidence)

# Save routing table
routing_file = project_root / f'route_model_routing_{TIMESTAMP}.csv'
routing_table.to_csv(routing_file, index=False)
print(f"💾 Saved: {routing_file.name}")

# Also save as current routing table
current_routing_file = project_root / 'route_model_routing_table.csv'
routing_table.to_csv(current_routing_file, index=False)
print(f"💾 Updated: {current_routing_file.name}")

## Step 7: Update Performance Tracking Database

In [ ]:
# Save performance summary
performance_summary = {
    'timestamp': TIMESTAMP,
    'evaluation_week': EVALUATION_WEEK,
    'evaluation_year': EVALUATION_YEAR,
    'total_routes': len(df_forecasts),
    'total_actuals': int(df_forecasts['Actual'].sum()),
    'model_wins': win_summary.to_dict()
}

perf_file = project_root / f'model_performance_summary_{TIMESTAMP}.json'
with open(perf_file, 'w') as f:
    json.dump(performance_summary, f, indent=2)

print(f"💾 Saved: {perf_file.name}")
print(f"\n📊 Performance Summary:")
print(json.dumps(performance_summary, indent=2))

## Step 8: Generate Forecast for Next Week Using Winners

In [ ]:
# Load updated historical data for forecast week
df_historical_forecast = load_historical_data(conn, FORECAST_WEEK, FORECAST_YEAR, TABLE_NAME, years=4)

print(f"\n📊 Generating forecasts for week {FORECAST_WEEK} using optimal models...\n")

# Get routes to forecast (use routing table)
forecast_results = []

# Progress bar
for idx, route in tqdm(routing_table.iterrows(), total=len(routing_table), desc="Generating forecasts"):
    odc, ddc, product, dow = route['ODC'], route['DDC'], route['ProductType'], route['dayofweek']
    best_model = route['best_model']
    
    # Get route history
    route_data = df_historical_forecast[
        (df_historical_forecast['ODC'] == odc) &
        (df_historical_forecast['DDC'] == ddc) &
        (df_historical_forecast['ProductType'] == product) &
        (df_historical_forecast['dayofweek'] == dow)
    ].sort_values('date', ascending=False)
    
    # Find model function
    model_func = None
    for name, func in model_functions:
        if name == best_model:
            model_func = func
            break
    
    if model_func is None:
        forecast = 0
    else:
        try:
            forecast = model_func(route_data, FORECAST_WEEK, FORECAST_YEAR, product)
            forecast = max(0, forecast)
        except:
            forecast = 0
    
    # Calculate variance (use ±50% as default)
    variance_pct = 50.0
    variance_pieces = forecast * (variance_pct / 100)
    
    forecast_results.append({
        'route_key': route['route_key'],
        'ODC': odc,
        'DDC': ddc,
        'ProductType': product,
        'dayofweek': dow,
        'week_number': FORECAST_WEEK,
        'year': FORECAST_YEAR,
        'forecast': forecast,
        'optimal_model': best_model,
        'confidence': route['confidence'],
        'historical_error_pct': route['best_error'],
        'forecast_low': max(0, forecast - variance_pieces),
        'forecast_high': forecast + variance_pieces,
        'variance_pieces': variance_pieces,
        'variance_pct': variance_pct
    })

df_production_forecast = pd.DataFrame(forecast_results)
print(f"\n✅ Generated {len(df_production_forecast):,} forecasts for week {FORECAST_WEEK}")
print(f"   Total forecast: {df_production_forecast['forecast'].sum():,.0f} pieces")

## Step 9: Save Production Forecast

In [ ]:
# Save production forecast
prod_forecast_file = project_root / f'production_forecast_week{FORECAST_WEEK}.csv'
df_production_forecast.to_csv(prod_forecast_file, index=False)
print(f"💾 Saved: {prod_forecast_file.name}")

# Show sample
print("\nSample forecast:")
display(df_production_forecast.head(10))

## Step 10: Summary Report

In [ ]:
print("="*80)
print(f"COMPREHENSIVE WEEKLY UPDATE COMPLETE - {TIMESTAMP}")
print("="*80)

print(f"\n📊 Evaluation (Week {EVALUATION_WEEK}, {EVALUATION_YEAR}):")
print(f"  • Routes evaluated: {len(df_forecasts):,}")
print(f"  • Actual pieces: {df_actuals['actual_pieces'].sum():,.0f}")
print(f"  • Models tested: {len(model_cols)}")

print(f"\n🏆 Top Performing Models:")
for i, (model, count) in enumerate(win_summary.head(5).items(), 1):
    pct = count / len(df_forecasts) * 100
    print(f"  {i}. {model}: {count:,} wins ({pct:.1f}%)")

print(f"\n📈 Forecast (Week {FORECAST_WEEK}, {FORECAST_YEAR}):")
print(f"  • Routes forecasted: {len(df_production_forecast):,}")
print(f"  • Total forecast: {df_production_forecast['forecast'].sum():,.0f} pieces")
print(f"  • Forecast range: {df_production_forecast['forecast_low'].sum():,.0f} - {df_production_forecast['forecast_high'].sum():,.0f}")

by_product = df_production_forecast.groupby('ProductType')['forecast'].sum()
print(f"\n  By Product Type:")
for product, total in by_product.items():
    print(f"    • {product}: {total:,.0f} pieces")

print(f"\n💾 Files Generated:")
print(f"  • {output_file.name}")
print(f"  • {routing_file.name}")
print(f"  • {current_routing_file.name}")
print(f"  • {perf_file.name}")
print(f"  • {prod_forecast_file.name}")

print(f"\n" + "="*80)
print("✅ ALL UPDATES COMPLETE!")
print("="*80)

# Close connection
conn.close()
print("\n🔌 Databricks connection closed")